In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work/Molecular_Properties/model-pipeline/src')
import pandas as pd
import warnings
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from model_construction import prepare_data_split, run_model, show_varimp
from bearinmind_pipeline.data_preprocessing import dataPreprocessing as dp
#from bearinmind_pipeline.model_construction import modelBuilder as mb
import bearinmind_pipeline.model_construction as mc
#from bearinmind_pipeline import data_preprocessing, model_construction, results_blend
import gc
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import os
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from fbprophet import Prophet
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)
from multiprocessing import Pool, cpu_count
import time


Using TensorFlow backend.


In [2]:
import datetime
from tqdm import tqdm_notebook, tqdm 

In [3]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
mapping = pd.read_csv('data/building_metadata.csv')
weather_train = pd.read_csv('data/weather_train.csv')
weather_test = pd.read_csv('data/weather_test.csv')

In [4]:
df_train = df_train.merge(mapping, how = "left", on = "building_id")

In [5]:
df_test = df_test.merge(mapping, how = "left", on = "building_id")

In [6]:
df_train = df_train.merge(weather_train, how = "left", on = ["site_id", "timestamp"])
df_test = df_test.merge(weather_test, how = "left", on =  ["site_id", "timestamp"])

In [7]:
def rmsle(y, yhat):
    score = np.sqrt(np.mean(np.power(np.log(yhat+1)-np.log(y.reset_index(drop=True)+1), 2)))
    return(score)

In [8]:
df_train["timestamp"] = pd.to_datetime(df_train["timestamp"])

In [9]:
df_test["timestamp"] = pd.to_datetime(df_test["timestamp"])

In [10]:
df_train["air_temperature"] = df_train["air_temperature"].fillna(df_train["air_temperature"].mean())
df_test["air_temperature"] = df_test["air_temperature"].fillna(df_test["air_temperature"].mean())
df_train["dew_temperature"] = df_train["dew_temperature"].fillna(df_train["dew_temperature"].mean())
df_test["dew_temperature"] = df_test["dew_temperature"].fillna(df_test["dew_temperature"].mean())
df_train["sea_level_pressure"] = df_train["sea_level_pressure"].fillna(df_train["sea_level_pressure"].mean())
df_test["sea_level_pressure"] = df_test["sea_level_pressure"].fillna(df_test["sea_level_pressure"].mean())
df_train["wind_direction"] = df_train["wind_direction"].fillna(df_train["wind_direction"].mean())
df_test["wind_direction"] = df_test["wind_direction"].fillna(df_test["wind_direction"].mean())
df_train["wind_speed"] = df_train["wind_speed"].fillna(df_train["wind_speed"].mean())
df_test["wind_speed"] = df_test["wind_speed"].fillna(df_test["wind_speed"].mean())

In [14]:
df_train['it_key'] = df_train.groupby(['building_id','meter']).ngroup()

In [18]:
df_test['it_key'] = df_test.groupby(['building_id','meter']).ngroup()

In [20]:
def run_prophet(iter_idx):
    #for i in build_idx:
        #scores = []
        #df_train_build = df_train[df_train["it_key"] == iter_idx]
        #df_test_build = df_test[df_test["it_key"] == iter_idx]

        df_train_build_meter = df_train[df_train["it_key"] == iter_idx][["timestamp", "meter_reading", 
                                                                                 "air_temperature", 
                                                                                 "dew_temperature", 
                                                                                 "sea_level_pressure",
                                                                                 "wind_direction", 
                                                                                 "wind_speed"]].reset_index(drop=True)
        df_test_build_meter = df_test[df_test["it_key"] == iter_idx][["row_id","timestamp",
                                                                             "air_temperature", 
                                                                             "dew_temperature", 
                                                                             "sea_level_pressure",
                                                                             "wind_direction", 
                                                                             "wind_speed"]]

        df_train_build_meter.columns = ["ds", "y", "air_temperature", "dew_temperature",
                                                                        "sea_level_pressure",
                                                                        "wind_direction", 
                                                                        "wind_speed"]
        df_test_build_meter.columns = ["row_id", "ds", "air_temperature", "dew_temperature", 
                                                                              "sea_level_pressure",
                                                                              "wind_direction", 
                                                                              "wind_speed"]

        z = Prophet(interval_width=0.95, daily_seasonality=True, weekly_seasonality=True,
             seasonality_mode = 'multiplicative')
        z.add_regressor('air_temperature')
        z.add_regressor('dew_temperature')
        z.add_regressor('sea_level_pressure')
        z.add_regressor('wind_direction')
        z.add_regressor('wind_speed')
        z.add_seasonality(name='hourly', period=24, fourier_order=2)
        z.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        z.fit(df_train_build_meter)

        submit = z.predict(df_test_build_meter[["ds", "air_temperature", 
                                   "dew_temperature", 
                                   "sea_level_pressure",
                                   "wind_direction", 
                                   "wind_speed"
                                  ]])

        #submit_pred = fit_model.predict(X_test)
        submit= pd.DataFrame(submit["yhat"])
        submit.columns = ['meter_reading']
        submit = pd.concat([df_test_build_meter['row_id'].reset_index(drop=True), submit], axis = 1)
        #submit_pred.append(submit)
           
        #gc.collect()
            
        print(f'Building {iter_idx} is finished ...')
            
        #del df_train_build_meter
        #del df_test_build_meter
        
        return(submit)

In [21]:
dict_scores = {}
submit_pred = []
#test_build_idx = range(5)
#build_idx = list(df_train["building_id"].unique())
iter_idx = list(df_train["it_key"].unique())

In [22]:
gc.collect()

697

In [ ]:
#cpu = int(cpu_count()/3)
start_time = time.time()
cpu = 32

p = Pool(cpu)
predictions = list(tqdm(p.imap(run_prophet, iter_idx), total=len(iter_idx)))
p.close()
p.join()

print("--- %s seconds ---" % (time.time() - start_time))

  0%|          | 0/2380 [00:00<?, ?it/s]

Building 5 is finished ...
Building 27 is finished ...
Building 41 is finished ...
Building 7 is finished ...
Building 31 is finished ...
Building 10 is finished ...
Building 30 is finished ...
Building 29 is finished ...
Building 12 is finished ...
Building 26 is finished ...
Building 9 is finished ...
Building 15 is finished ...
Building 22 is finished ...
Building 39 is finished ...
Building 33 is finished ...
Building 17 is finished ...
Building 24 is finished ...
Building 14 is finished ...
Building 6 is finished ...
Building 19 is finished ...
Building 37 is finished ...
Building 32 is finished ...
Building 28 is finished ...
Building 3 is finished ...
Building 2 is finished ...
Building 1 is finished ...
Building 25 is finished ...
Building 0 is finished ...


  0%|          | 1/2380 [11:15<446:22:29, 675.47s/it]

Building 4 is finished ...


  0%|          | 5/2380 [11:15<311:57:33, 472.86s/it]

...

 69%|██████▉   | 1637/2380 [9:58:42<9:05:52, 44.08s/it]

Building 1703 is finished ...
Building 1702 is finished ...


 69%|██████▉   | 1639/2380 [9:59:17<7:25:25, 36.07s/it]

Building 1705 is finished ...
Building 1707 is finished ...
Building 1704 is finished ...


 69%|██████▉   | 1641/2380 [10:00:26<7:17:40, 35.53s/it]

In [ ]:
final_submit = pd.concat(predictions, axis = 0).reset_index(drop = True)

In [ ]:
final_submit = final_submit.sort_values(by=['row_id'])

In [ ]:
final_submit.to_csv("submission_prophet.csv", index = False)

In [28]:
!kaggle competitions submit -c ashrae-energy-prediction -f submission_prophet.csv -m "Prophet with whether data and seasonality"

100%|██████████████████████████████████████| 1.07G/1.07G [00:22<00:00, 50.7MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III

In [27]:
final_submit

,row_id,meter_reading
0,0,155.096
17520,1,87.233
35040,2,-0.940
52560,3,251.647
70080,4,1223.351
...,...,...
40243439,41697595,-20.326
40260959,41697596,16.497
40278479,41697597,-35.076
40295999,41697598,-347.733


In [ ]:
########## Experiment ###############

In [306]:
### Loop to run on all buildings and meters ###
start_time = time.time()
dict_scores = {}
submit_pred = []
build_idx = df_train["building_id"].unique()
for i in tqdm_notebook(test_build_idx):
    scores = []
    df_train_build = df_train[df_train["building_id"] == i]
    df_test_build = df_test[df_test["building_id"] == i]

    #print(f'building {i} is being trained ...')
    for j in df_train_build["meter"].unique():
        #print(f'Jumping to {j} meter ...')
        
        df_train_build_meter = df_train_build[df_train_build["meter"] == j][["timestamp", "meter_reading", 
                                                                             "air_temperature", 
                                                                             "dew_temperature", 
                                                                             "sea_level_pressure",
                                                                             "wind_direction", 
                                                                             "wind_speed"]].reset_index(drop=True)
        df_test_build_meter = df_test_build[df_test_build["meter"] == j][["row_id","timestamp",
                                                                         "air_temperature", 
                                                                         "dew_temperature", 
                                                                         "sea_level_pressure",
                                                                         "wind_direction", 
                                                                         "wind_speed"]]
        
        df_train_build_meter.columns = ["ds", "y", "air_temperature", "dew_temperature",
                                                                    "sea_level_pressure",
                                                                    "wind_direction", 
                                                                    "wind_speed"]
        df_test_build_meter.columns = ["row_id", "ds", "air_temperature", "dew_temperature", 
                                                                          "sea_level_pressure",
                                                                          "wind_direction", 
                                                                          "wind_speed"]

        z = Prophet(interval_width=0.95, daily_seasonality=True, weekly_seasonality=True,
           seasonality_mode = 'multiplicative')
        z.add_regressor('air_temperature')
        z.add_regressor('dew_temperature')
        z.add_regressor('sea_level_pressure')
        z.add_regressor('wind_direction')
        z.add_regressor('wind_speed')
        z.add_seasonality(name='hourly', period=24, fourier_order=2)
        z.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        z.fit(df_train_build_meter)
        
        submit = z.predict(df_test_build_meter[["ds", "air_temperature", 
                               "dew_temperature", 
                               "sea_level_pressure",
                               "wind_direction", 
                               "wind_speed"
                              ]])
        
        #submit_pred = fit_model.predict(X_test)
        submit= pd.DataFrame(submit["yhat"])
        submit.columns = ['meter_reading']
        submit = pd.concat([df_test_build_meter['row_id'].reset_index(drop=True), submit], axis = 1)
        submit_pred.append(submit)

print("--- %s seconds ---" % (time.time() - start_time))

--- 129.63623976707458 seconds ---


In [321]:
final_submit = pd.concat(submit_pred, axis = 0).reset_index(drop = True)

In [322]:
final_submit['meter_reading'].isnull().sum()

0

In [323]:
final_submit

,row_id,meter_reading
0,0,155.096
1,129,157.546
2,258,155.900
3,387,157.603
4,516,156.628
...,...,...
35035,2259436,33.147
35036,2259565,31.234
35037,2259694,31.262
35038,2259823,31.239
